<h4>(1 - Preferred). Create and save a new notebook to enable automatic re-mounting of drive storage. Copy and paste the below code into the new notebook and follow the directions in the <em>How To Use</em> section of the <em>README</em>: <a href="http://colab.research.google.com/#create=true" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4><h4>(2). Open the pre-formed version of Colab (requires manual authorization each time a notebook is opened) <a href="https://colab.research.google.com/github/Danc2050/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h4>

In [ ]:
#@title <strong> <h1>📔 Colab GPU and CPU Notebook</strong></h1> 
#{ output-height: 8, form-width: "96%", display-mode: "form" }

#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.</h4>
#@markdown <h4>🔌 Make sure the GPU is enabled under <em>Runtime→Change runtime type</em></h4>
#@markdown <h4>💡 Keep each notebook <strong>open</strong> in the browser or 📌 the tab to a <a href="https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open" target="_blank">dedicated window</a> to prevent disconnection.</h4>
#@markdown <h4>🕷️ A <em>debug</em> field outputs CUDALucas or Prime95 progress + GPU specs, then exits.</h4>
#@markdown <h4>📜 See the <a href="https://github.com/Danc2050/Distributed-Computing-Scripts/tree/master/google-colab" target="_blank">documentation</a> for more information.</h4>
#@markdown <h4>🤷‍♂️ Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/" target="_blank">here</a>.</h4>

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
prime_password = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
GPU_type_of_work = '100 - First time LL tests' #@param ['100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests']
CPU_type_of_work = '100 - First time LL tests' #@param ['0 - Whatever makes the most sense', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '154 - 100 million digit PRP tests']
computer_number = "Default (1)" #@param ["Default (1)", "2"] {allow-input: true}
output_type = 'GPU (CUDALucas)' #@param ['GPU (CUDALucas)', 'CPU (Prime95)'] 
debug = 'False' #@param ['False', 'GPU (CUDALucas)', 'CPU (Prime95)']
gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
path_dir = "" # helps us to not %cd in the optimize_gpu function

import os

class StopExecution(Exception):
  def _render_traceback_(self):
      pass

def optimize_gpu():
  '''If a new GPU is being used, optimize CUDALucas for using this GPU'''
  print('\nOptimizing CUDALucas for this computer and GPU\n')
  !cd cudalucas && chmod 777 CUDALucas
  if not os.path.exists(path_dir + gpu_info + ' fft.txt'):
    !cd cudalucas && ./CUDALucas -cufftbench 1024 8192 5
  if not os.path.exists(path_dir + gpu_info + ' threads.txt'):
    !cd cudalucas && ./CUDALucas -threadbench 1024 8192 5 0

def run():
  '''Run CUDALucas and Mprime'''
  print('\nStarting PrimeNet\n')
  optimize_gpu()
  !cd cudalucas; nohup python3 primenet.py -d -T $GPU_type_of_work -u $prime_ID -p $prime_password -i "{'worktodo' + computer_number + '.txt'}" &
  !sleep 1
  while not os.path.exists('cudalucas/worktodo' + computer_number+ '.txt'):
    print(f'Waiting for worktodo{computer_number}.txt access...')
    !sleep 1 

  if output_type == 'GPU (CUDALucas)':
    print('\nStarting Prime95\n')
    !cd mprime_gpu && chmod 777 mprime; nohup ./mprime -A$computer_number -d >> "{'cpu' + computer_number + '.out'}" &
    !cd cudalucas && ./CUDALucas -i "{'CUDALucas' + computer_number + '.ini'}" | tee -a "{'gpu' + computer_number + '.out'}"
  else:
    print('\nStarting CUDALucas\n')
    !cd cudalucas; nohup ./CUDALucas -i "{'CUDALucas' + computer_number + '.ini'}" >> "{'gpu' + computer_number + '.out'}" &
    !cd mprime_gpu && chmod 777 mprime && ./mprime -A$computer_number -d | tee -a "{'cpu' + computer_number + '.out'}"

def install():
  '''Download/Install/Configure CUDALucas then Prime95'''
  !wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
  print(gpu_info) # Output what GPU is assigned to this Notebook
  !wget -N https://github.com/Danc2050/Distributed-Computing-Scripts/archive/master.zip -nv
  !unzip -o master.zip
  
  !sed -i '/^[[:blank:]]*if ! COMPUTE=/,/^[[:blank:]]*fi/!b; /^[[:blank:]]*fi/a echo "$COMPUTE"' Distributed-Computing-Scripts-master/cudalucas2.sh # Output CUDA compute capability of GPU
  !sed -i 's/\/$COMPUTE/\/--generate-code arch=compute_35,code=sm_35 --generate-code arch=compute_50,code=sm_50 --generate-code arch=compute_60,code=sm_60 --generate-code arch=compute_70,code=sm_70 --generate-code arch=compute_75,code=sm_75/' Distributed-Computing-Scripts-master/cudalucas2.sh
  !sed -i '/^GPU=/,/^fi/ s/^/# /' Distributed-Computing-Scripts-master/cudalucas2.sh # Do not check for an Nvidia GPU
  !sed -i '/^nohup / s/^/# /' Distributed-Computing-Scripts-master/cudalucas2.sh # Do not start PrimeNet
  !sed -i '/^crontab / s/^/# /' Distributed-Computing-Scripts-master/cudalucas2.sh # Do not create a cronjob
  !sed -i '/^\.\/CUDALucas / s/^/# /' Distributed-Computing-Scripts-master/cudalucas2.sh # Disable optimization step for faster install
  !sudo apt-get install gawk -y # cudalucas2.sh relies on gawk
  !bash -- Distributed-Computing-Scripts-master/cudalucas2.sh $computer_number "" $prime_ID $GPU_type_of_work # Do not pass any additional parameters here
  !cp -u Distributed-Computing-Scripts-master/google-colab/gpu_optimizations/* cudalucas/
  # TODO -- remove below after PR is merged.
  !cp Distributed-Computing-Scripts-master/primenet.py cudalucas/

  !sed -i '/^nohup / s/^/# /' Distributed-Computing-Scripts-master/mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' Distributed-Computing-Scripts-master/mprime2.sh # Do not create a cronjob
  !sed -i 's/"mprime"/"mprime_gpu"/' Distributed-Computing-Scripts-master/mprime2.sh # Name the folder specific to the runtime type
  !bash -- Distributed-Computing-Scripts-master/mprime2.sh $computer_number $prime_ID $computer_name $CPU_type_of_work # Run script
  run()

def debug_exit():
  '''Output GPU and output of Prime95 or CUDALucas output'''
  if debug == 'GPU (CUDALucas)'and os.path.exists('cudalucas/gpu' + computer_number + '.out'):
    !echo -e "\nOutput for computer number {computer_number}:\n"
    !echo -e "\nPrimenet output:\n" && tail -n 100 "cudalucas/nohup.out" # view primenet output
    !echo -e "\nGPU (CUDALucas) output: " && tail -n 100 "{'cudalucas/gpu' + computer_number + '.out'}"; echo # view CUDALucas progress
  elif debug == 'CPU (Prime95)' and os.path.exists('mprime_gpu/cpu' + computer_number + '.out'):
    !echo -e "\nCPU (Prime95) output:\n" && tail -n 100 "{'mprime_gpu/cpu' + computer_number + '.out'}"; echo # view mprime progress
  else:
    print(f'No `{debug}` output file found for debug option and computer number `{computer_number}`.\n')

def load_drive():
  '''Load & cd into gdrive for persistent data'''
  global path_dir
  if os.getcwd() != "/content/drive/My Drive/GIMPS": # don't create a subfolder in GIMPS/
      !mkdir -p GIMPS
  if os.path.exists("/content/gdrive/My Drive"): # use our notebook
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd "/content/gdrive/My Drive/GIMPS"
    path_dir = "/content/gdrive/My Drive/GIMPS/cudalucas/"
  elif os.path.exists("/content/drive/My Drive"): # create your own notebook with our code
    %cd "/content/drive/My Drive/GIMPS"
    path_dir = "/content/drive/My Drive/GIMPS/cudalucas/"


def gpu_check():
  '''GPU Check'''
  global gpu_info
  gpu_info = "\n".join(gpu_info)
  if gpu_info.find('failed') >= 0:
    return False
  return True

if not gpu_check():
  print('Select the "Runtime" → "Change runtime type" → "GPU" → "SAVE" to enable a GPU accelerator, ')
  print('and then re-execute this cell.')

load_drive()

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
prime_password = 'vikings' if prime_password.lower() == 'default' else prime_password
computer_number = "1" if computer_number.lower() == 'default (1)' else computer_number
CPU_type_of_work = CPU_type_of_work.split("-")[0].rstrip()
GPU_type_of_work = GPU_type_of_work.split("-")[0].rstrip()
debug = False if debug == 'False' else debug

# Add quotes to string args so script can parse spaces/special characters
prime_ID = f'"'+ prime_ID + f'"'
computer_name = f'"'+ computer_name + f'"'
prime_password = f'"'+ prime_password + f'"'

if debug:
  debug_exit()
  raise StopExecution

elif not computer_number.isdigit():
  print("ERROR: Computer number must be a number")
  raise StopExecution

elif os.path.exists('mprime_gpu/work000' + computer_number + '.txt') and os.path.exists('cudalucas/CUDALucas' + computer_number + '.ini'):
  run()

else:
  install()

print("Gracefully exiting...")